In [ ]:
# make sure mongodb is running

In [ ]:
!pip install katna

In [ ]:
import os
from Katna.video import Video
from pymongo import MongoClient
import multiprocessing
import multiprocessing.pool
import cv2
import re
import utils.const as const

In [ ]:
poolSize=8

### most important
frameToExtract=10

###### make queue for each process
failed=multiprocessing.Queue()
extracted=multiprocessing.Queue()
###### make queue for each process

### katna will quickly fill up the ram so we will do keyframe extraction using multiple processes in batch
bigBatch=100 

db=const.grpId
collection=const.PostCollection
directory=db

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
## read logs for already extracted keyframes and create dictionary

alreadyExtracted={}

regex = re.compile('(extracted*.*txt$)')

for file in os.listdir():
  if regex.match(file):
    with open(file,'r') as fp:
      for line in fp.readlines():
        print(line[:-1])
        alreadyExtracted[line[:-1]]=1

In [ ]:
vids=[]
for post in myCollect.find({}):
  if '_id' in post:
    A=os.path.join(directory,post['_id'])

    if 'attachments' in post:
      B=os.path.join(A,'attachments')

      if 'videos' in post['attachments']:
        C=os.path.join(B,'videos')
        for i in range(len(post['attachments']['videos'])):
          D=os.path.join(C,str(i))
          vids.append(D)
      
          
    
    if 'comments' in post:
      B=os.path.join(A,'comments')
      for comment in post['comments']:
        if '_id' in comment:
          C=os.path.join(B,comment['_id'])

          if '_id' and 'attachment' in comment:
            D=os.path.join(C,'attachment')

            if 'videos' in comment['attachment']:
              E=os.path.join(D,'videos')

              for i in range(len(comment['attachment']['videos'])):
                F=os.path.join(E,str(i))
                vids.append(F)


print(len(vids))
print(len(alreadyExtracted))

In [ ]:
order=[]

if os.path.isfile('order.txt'):
    print('order.txt exist')
    with open('order.txt','r') as fp:
        for line in fp.readlines():
            if len(line)<=1:
                continue
            order.append(line[:-1])
else:
    print('creating order.txt')
    dictOrder={}
    totalSize=0
    maxSize=0
    for i in vids:
        dictOrder[i]=os.stat(i).st_size
        totalSize+=dictOrder[i]
        maxSize=max(maxSize,dictOrder[i])
    
    print('Max size',maxSize)
    print('Total size',totalSize)
    
    dict2=dict(sorted(dictOrder.items(), key=lambda item: item[1]))

    with open('order.txt','w') as fp:
        for k,v in dict2.items():
            order.append(k)
            fp.write(k+'\n')

In [ ]:
vids=[]
for i in order:
    if i not in alreadyExtracted:
        vids.append(i)
print(len(vids))

In [ ]:
class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [ ]:
def worker(low,high,ind):
  vd = Video()
  size=high-low
  fp=open('extracted'+str(ind)+'.txt','a')

  print(ind,low,high)
  for i in range(low,high):
    videoPath=vids[i]
    data_output_dir=vids[i]+'_key_frames'

    #print('A')
    cap = cv2.VideoCapture(videoPath)

    if not cap.isOpened(): 
      print (ind,"could not open :",videoPath)
      failed.put(vids[i])
      continue
    print(ind,'B')
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(ind,'frames '+str(frames))

    
    try:
      
      if os.path.isdir(data_output_dir):
        if len(os.listdir(data_output_dir)) == min(frames,frameToExtract):
          extracted.put(vids[i])
          fp.write(vids[i]+'\n')
          continue
      else:
        os.mkdir(data_output_dir)
      
      print(ind,'D')

      if frames>frameToExtract:
        imgs = vd.extract_video_keyframes(no_of_frames = frameToExtract, file_path=videoPath )
        
        for counter,img in enumerate(imgs):
          vd.save_frame_to_disk(img, file_path=data_output_dir, file_name=str(counter), file_ext=".png")
      
      else:
        success,image = cap.read()
        counter = 0
        while success:
          cv2.imwrite(os.path.join(data_output_dir,str(counter))+'.png', image)
          counter+=1     
          success,image = cap.read()

      #print('E')
      cap.release()
      extracted.put(vids[i])
      fp.write(vids[i]+'\n')
      #print('F')


    except Exception as e:
      print(ind,str(e))
      print (ind,"could not extract key frame from :"+videoPath + " with "+str(cap.get(cv2.CAP_PROP_FRAME_COUNT))+" frames")
      failed.put(vids[i])

    print(ind,(i+1-low)/size*100)

  fp.close()

In [ ]:
iter=int(len(vids)/bigBatch)
while iter*bigBatch<len(vids):
  iter+=1

print(iter)

In [ ]:
while not failed.empty():
  failed.get() 
while not extracted.empty():
  extracted.get() 
for i in range(iter):
  low,high=i*bigBatch,min(len(vids),(i+1)*bigBatch)
  size=high-low+1
                       
  batch=int((size)/poolSize)
  while batch*poolSize<size:
    batch+=1
  print(str(batch)+' for iter '+str(i)+' in range '+str(low)+' '+str(high))
  
  pool = MyPool(poolSize)
  for j in range(poolSize):
    pool.apply_async(worker, (j*batch +low ,min(high,(j+1)*batch+low), j ,))

  pool.close()
  pool.join()

In [ ]:
f=failed.qsize()
e=extracted.qsize()
print(f,e)

In [ ]:
if f:
    print('try  running 2nd notebook on key-frame')
else:
    print('done')